In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import pandas as pd
import bs4
import requests
from urllib.request import urlopen

In [4]:
# 각 섹션별 링크주소 파일 가져오기
df_menu = pd.read_csv('./crawl_data/naver_news_section.csv', index_col=0)
df_menu.head()

,section,link
0,정치,https://news.naver.com/main/main.naver?mode=LS...
1,경제,https://news.naver.com/main/main.naver?mode=LS...
2,사회,https://news.naver.com/main/main.naver?mode=LS...
3,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
4,IT/과학,https://news.naver.com/main/main.naver?mode=LS...


### 기존 코드(02_1) 내용을 결합해서 함수로 구성
- 다른 섹션에도 적용하기 위해
- 함수명 : get_topic(url, section)
- 반환 : 섹션별 수집 데이터를 dict로 구성 후 반환
<hr>
- 함수가 반환해 준 dict를 df로 구성 후에 기존 df에 모두 결합해서 df 완성
    - 6개의 섹션의 헤드라인을 한 파일로 구성

In [12]:
# 함수 구성
def get_topic(url, section):
    # 전달된 url의 소스 요청하는 코드
    res = urlopen(url)
    html = res.read()
    # 요청 후 반환된 소스 bs 객체로 생성
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')
    # headline 뉴스 전체 추출
    # class : sh_text
    head_line = bs_obj.findAll('div', {'class':'sh_text'})
    
    # 빈 리스트 생성 => 수집한 각 data를 담을 빈 리스트
    headline_title = []
    headline_link = []
    headline_desc = []
    
    # for문으로 head_line 뉴스들의 data를 가져오기
    # 각 headline 기사의 세부내용 추출 후 list에 추가하는 코드
    for head in head_line:
        try:
            a_t = head.find('a')
            div_t = head.find('div',{'class':'sh_text_lede'})
            headline_title.append(a_t.text)
            headline_link.append(a_t['href'])
            headline_desc.append(div_t.text)
        except:
            print('에러발생')
    # 함수의 return 값으로 수집된 data를 dict로 생성 후 반환        
    return ({'기사제목': headline_title, 
            '기사링크': headline_link, 
            '기사요약': headline_desc,
             '분류' : section,
              })

In [9]:
# test url 이용해서 함수 test
url = df_menu.loc[1].link
url

'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'

In [13]:
# 함수 테스트 후 df 생성
pd.DataFrame(get_topic(url, '경제'))

,기사제목,기사링크,기사요약,분류
0,원희룡 “건설현장서 수십년 넘게 쌓인 이권 카르텔 도려내겠다”,https://n.news.naver.com/mnews/article/082/000...,원희룡 국토교통부 장관이 건설 분야에서 부정한 ‘이권 카르텔’에 대해 엄정하게 대처...,경제
1,"뉴욕증시, 연준 선호 물가 둔화에 강세…나스닥 1.9 %↑[3분증시]",https://n.news.naver.com/mnews/article/422/000...,세계 증시와 우리 증시를 짚어보는 시간 '3분 증시'입니다. 연합인포맥스 권용욱 기...,경제
2,"한국타이어, 2Q 실적 호조…체질 대비 지나친 저평가-신한",https://n.news.naver.com/mnews/article/018/000...,신한투자증권은 31일 한국타이어앤테크놀로지(161390)에 대해 올 2분기 호실적을...,경제
3,"'긴축' 페달에 발 올린 일본은행, 가속은 언제? [김경민의 도쿄 혼네]",https://n.news.naver.com/mnews/article/014/000...,일본은행(BOJ)이 7개월 만에 또 다시 미세 긴축에 나서면서 국제 금융 시장이 요...,경제
4,"""플립5 사면 샤넬 레인부츠 추첨""…LGU+, 갤Z5 사전예약 실시",https://n.news.naver.com/mnews/article/014/000...,LG유플러스는 삼성전자 갤럭시Z플립5·폴드5 사전예약 가입을 오는 8월 1일부터 7...,경제
5,"中 배터리 기업들, 한국에 판 벌렸다…5조 쏟아부은 속내",https://n.news.naver.com/mnews/article/015/000...,이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. 올...,경제
6,2분기 카드 승인액 해외여행 증가에 292조…전년比 4.1%↑,https://n.news.naver.com/mnews/article/119/000...,올해 2분기 카드 사용액이 내수 회복에 이어 여행‧여가 관련 산업 매출 영향으로 증...,경제
7,日 방문 한국인 313만 명...한국 방문 일본인의 3.6배,https://n.news.naver.com/mnews/article/052/000...,올해 상반기 일본을 찾은 한국인 수가 300만 명을 넘어 한국을 방문한 일본인의 3...,경제
8,"원희룡 ""양평고속道 노선평가위에 협조하겠다…국정조사 갈 것도 없어""",https://n.news.naver.com/mnews/article/421/000...,원희룡 국토교통부 장관이 김건희 여사 일가 특혜 의혹이 불거진 서울~양평 고속도로 ...,경제
9,"셀트리온, 아일리아 시밀러 'CT-P42' 임상 3상 결과 공개",https://n.news.naver.com/mnews/article/277/000...,셀트리온이 안과 질환 치료제인 '아일리아(성분명 애플리버셉트)'의 바이오시밀러(바이...,경제


### 모든 섹션에 함수 적용 후 df로 생성하고 csv 파일로 저장
0. 빈 df 생성 후
1. 반복문을 통해 아래 작업 진행
    1. 반환되는 각 섹션의 dict를 temp df로 생성 후
    2. 기존 저장 df 와 결합

In [14]:
# 0. 빈 df 생성 후
dict_sub = {}
dict_sub['기사제목'] = []
dict_sub['기사링크'] = []
dict_sub['기사요약'] = []
dict_sub['분류'] = []

headline_df = pd.DataFrame(dict_sub)
headline_df

,기사제목,기사링크,기사요약,분류


In [17]:
# 1. 반복문을 통해 아래 작업 진행
#     1. 반환되는 각 섹션의 dict를 temp df로 생성 후
#     2. 기존 저장 df 와 결합

# df_menu['link'] # 0번행부터 5번행까지
# df_menu['section'] # 0번행부터 5번행까지

for i in range(0,6):
    
    temp = pd.DataFrame(get_topic(df_menu['link'][i], df_menu['section'][i]))
    headline_df = pd.concat([headline_df, temp], axis=0, ignore_index=True)
    # ignore_index=True => 붙일때마다 인덱스 다시 만드는것

In [18]:
headline_df.head()
headline_df.tail()

,기사제목,기사링크,기사요약,분류
0,"文 “단 한 건도 금품 부정비리 없었던 청와대, 고마워”",https://n.news.naver.com/mnews/article/005/000...,문재인 전 대통령이 재임시절을 돌아보며 “단 한 건도 금품과 관련된 부정비리가 없었...,정치
1,美 별장서 모이는 한미일…북중러 밀착에 협력 수위 높이나,https://n.news.naver.com/mnews/article/015/000...,"윤석열 대통령과 조 바이든 미국 대통령, 기시다 후미오 일본 총리가 다음달 18일 ...",정치
2,"‘개봉영화‘ 공짜로 보는 이곳, 없어질까…당정, 누누티비 등 불법 스트리밍 방지대책 논의",https://n.news.naver.com/mnews/article/021/000...,"당정, K-콘텐츠 불법유통 ‘누누티비’ 방지대책 논의 해외 공조 등 불법 유통 차단...",정치
3,"민주당 ""쌍방울-친윤 법조 카르텔"" 비난…수사 검사 4명 실명 공개",https://n.news.naver.com/mnews/article/055/000...,민주당은 오늘(30일) 김성태 전 쌍방울 회장 관련 '봐주기 기소' 의혹을 제기하며...,정치
4,尹지지율 0.7%p 오른 37.3%...국힘 36.3% 민주 44.3%[리얼미터],https://n.news.naver.com/mnews/article/009/000...,윤석열 대통령의 국정 수행에 대한 긍정 평가가 하락세를 멈추며 4주 만에 반등했다는...,정치


,기사제목,기사링크,기사요약,분류
49,"세계 각국, 쿠데타 발생 니제르에 원조 중단…제재도 '만지작'",https://n.news.naver.com/mnews/article/421/000...,최근 쿠데타가 발생한 서아프리카 국가 니제르에 대한 원조 중단이 잇따르는 가운데 제...,세계
50,"폴란드 총리 ""바그너 용병들 우리 국경 근처로 이동…침투 우려""",https://n.news.naver.com/mnews/article/055/000...,벨라루스에 있는 러시아 바그너 그룹 용병들이 폴란드의 전략적 요충지와 인접한 국경 ...,세계
51,"美경찰견 8마리 차 안에서 '떼죽음'...""33도 넘는데 에어컨 고장""",https://n.news.naver.com/mnews/article/014/000...,"극심한 폭염이 지속되는 7월, 미국에서 에어컨이 고장 난 차량에 실려 훈련 시설로 ...",세계
52,"‘비밀 호수’에 회갈색 선체… 우크라, 자체 개발 해상 드론 첫 공개",https://n.news.naver.com/mnews/article/023/000...,우크라이나가 러시아를 견제하기 위해 자체 개발한 해상 드론을 서방 언론에 처음으로 ...,세계
53,"伊 국방장관 ""中일대일로 참여 결정은 형편없는 행동""",https://n.news.naver.com/mnews/article/052/000...,이탈리아가 중국의 일대일로 사업 탈퇴를 검토하는 가운데 내각에서도 일대일로에 대한 ...,세계


In [19]:
# 파일로 저장
headline_df.to_csv('./crawl_data/naver_news_headline.csv')